In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import os
import glob
import locale
import time
import re
import warnings
import openpyxl

from datetime import datetime
from urllib.parse import quote
from dateutil.relativedelta import relativedelta
from IPython.display import Markdown as md
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
from datetime import datetime, timedelta

In [2]:
try:
    FINnet_2_engine=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/FINCORE_BRIDGE_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))
except:
    print("Unable to create the Fincore Bridge DB connection")
try:
    FINnet_2_enginee=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/FINCORE_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))
except:
    print("Unable to create the Fincore Bridge DB connection")
try:
    DATAHUB_engine=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/DATAHUB_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))
except:
     print("Unable to create the DATAHUB DB connection")

In [3]:
x = int(input("Enter the Year and Month in the Format YYYYMM : "))
print('Entered Year and Month is :' , x)

Enter the Year and Month in the Format YYYYMM : 202403
Entered Year and Month is : 202403


### Anomalies

In [66]:
query_anomalie = f'''


with cte1 as (
select coalesce(oldreid,newreid) as FIUREID ,Upper(trim(re.name)) as RE_name ,COUNT(distinct concat(Transactionid,'|',reportId )) as Distinct_count  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
where left(Fiu_Batch_Id,6) = '{x}' and ts.reportType = 'BDU' and cp.Report_Status = 'PASS' and state_ in ('14','0')
group by coalesce(oldreid,newreid) ,Upper(trim(re.name))
),

cte2 as (


---Name Anomalie
select upper(trim(Re.name)) as RE_name,'Name' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.name is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

---Merchant Name Anomalie
select upper(trim(Re.name)) as RE_name,'Merchant Name' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.merchantName is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))


UNION

---Merchant Id
select upper(trim(Re.name)) as RE_name,'Merchant ID' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.merchantId is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION


---merchnatcountry
select upper(trim(Re.name)) as RE_name,'Merchant Country' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.merchantCountry is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))


UNION

--merchantposatmid
select upper(trim(Re.name)) as RE_name,'Merchant PoS/ATM ID' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.merchantPosAtmId is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--merchantcategorycode
select upper(trim(Re.name)) as RE_name,'Merchant Category Code' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.merchantCategoryCode is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))


UNION

--addressline1
select upper(trim(Re.name)) as RE_name,'Address Line 1' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.addressLine1 is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--locality
select upper(trim(Re.name)) as RE_name,'Locality' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.locality is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))


UNION

--pincode
select upper(trim(Re.name)) as RE_name,'Pincode' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.pinCode is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--state
select upper(trim(Re.name)) as RE_name,'State' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.state_ is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--country
select upper(trim(Re.name)) as RE_name,'Country' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.country is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--district
select upper(trim(Re.name)) as RE_name,'District' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.district is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--cityvillagetown
select upper(trim(Re.name)) as RE_name,'City/Village/Town' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.cityVillageTown is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--cardnumber
select upper(trim(Re.name)) as RE_name,'Card Number' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.cardNumber is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--ismasked
select upper(trim(Re.name)) as RE_name,'Masked / Non Masked' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.isMasked is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--issuingcountry
select upper(trim(Re.name)) as RE_name,'Issuing Country' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.issuingCountry is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--issuingbank
select upper(trim(Re.name)) as RE_name,'Issuing Bank' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.issuingBank is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--acquiring bank
select upper(trim(Re.name)) as RE_name,'Acquiring Bank' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.acquiringBank is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--transactiontype
select upper(trim(Re.name)) as RE_name,'Transaction Type' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.transactionType is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--amountinfc
select upper(trim(Re.name)) as RE_name,'Amount in FC' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.amountInFc is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--amountininr
select upper(trim(Re.name)) as RE_name,'Amount in INR' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.amountInInr is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--Txcurrencycode
select upper(trim(Re.name)) as RE_name,'Transaction Currency Code' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.txCurrencyCode is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--Transactiondate
select upper(trim(Re.name)) as RE_name,'Transaction Date' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.transactionDate is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--Transactiontime
select upper(trim(Re.name)) as RE_name,'Transaction Time' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.transactionTime is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name))

UNION

--TransactionID
select upper(trim(Re.name)) as RE_name,'Transaction ID' as 'Parameter' , 'Y' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
inner join cte1 on cte1.FIUREID = coalesce(re.oldreid,re.newreid)
where ts.reportType = 'BDU' and cp.Report_Status = 'PASS'  and left(cp.Fiu_Batch_Id,6) = '{x}'
and ts.transactionId is null and state_ in ('14','0')
group by cte1.Distinct_count , upper(trim(Re.name)) 

)

select upper(trim(Parameter)) as Parameter , RE_name , Mandatory , Distinct_count as Total_count , Missing_records , concat(cast((Missing_records*100 / Distinct_count) as double precision),'%') anomalies_percentage from cte2 order by Parameter 


'''




In [67]:
print(query_anomalie)




with cte1 as (
select coalesce(oldreid,newreid) as FIUREID ,Upper(trim(re.name)) as RE_name ,COUNT(distinct concat(Transactionid,'|',reportId )) as Distinct_count  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
where left(Fiu_Batch_Id,6) = '202403' and ts.reportType = 'BDU' and cp.Report_Status = 'PASS' and state_ in ('14','0')
group by coalesce(oldreid,newreid) ,Upper(trim(re.name))
),

cte2 as (


---Name Anomalie
select upper(trim(Re.name)) as RE_name,'Name' as 'Parameter' , 'N' as 'Mandatory' , cte1.Distinct_count , COUNT(distinct concat(Transactionid,'|',reportId )) as Missing_records  from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[F

In [68]:
query_anomalie = pd.read_sql(query_anomalie,FINnet_2_engine)
print('Query Executed successfully')
query_anomalie = pd.DataFrame(query_anomalie)
query_anomalie.columns = query_anomalie.columns.str.upper()
display("Shape : ",query_anomalie.shape)
display(query_anomalie.head(5))
query_anomalie.index = query_anomalie.index + 1
query_anomalie.head(5)

Query Executed successfully


'Shape : '

(16, 6)

,PARAMETER,RE_NAME,MANDATORY,TOTAL_COUNT,MISSING_RECORDS,ANOMALIES_PERCENTAGE
0,ADDRESS LINE 1,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
1,ADDRESS LINE 1,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
2,CITY/VILLAGE/TOWN,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
3,DISTRICT,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
4,DISTRICT,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%


,PARAMETER,RE_NAME,MANDATORY,TOTAL_COUNT,MISSING_RECORDS,ANOMALIES_PERCENTAGE
1,ADDRESS LINE 1,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
2,ADDRESS LINE 1,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
3,CITY/VILLAGE/TOWN,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
4,DISTRICT,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
5,DISTRICT,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%


### Raw Data

In [7]:
raw_data = f'''
SELECT distinct coalesce(re.oldreid,re.newreid) as FIUREID , re.name , concat('ID-',Fiu_Batch_Id) Fiu_Batch_Id , left(Fiu_Batch_Id,6) as YYYY_MM ,CONCAT('T-',transactionid) Transactionid, trim(upper(cou.name)) as Issuing_Country_name , trim(upper(st.name)) as State_Name , trim(upper(ct.name)) CityVillageTown_Name ,ts.*
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  INNER JOIN [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on cp.Re_Id = re.id_
  INNER JOIN [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven]  ts on cp.Report_Id = ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] cou on cou.id_ = ts.issuingCountry
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_State] st on st.id_ = ts.state_
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_City] ct on ct.id_ = ts.cityVillageTown
  where reportType = 'BDU' and left(Fiu_Batch_Id,6) = '{x}'
  and state_ in ('14','0')
  '''

In [8]:
raw_data = pd.read_sql(raw_data,FINnet_2_engine)
print('Query Executed successfully')
raw_data = pd.DataFrame(raw_data)
raw_data.columns = raw_data.columns.str.upper()
display("Shape : ",raw_data.shape)
display(raw_data.head(5))
raw_data.index = raw_data.index + 1
raw_data.head(5)

Query Executed successfully


'Shape : '

(7694, 68)

,FIUREID,NAME,FIU_BATCH_ID,YYYY_MM,TRANSACTIONID,ISSUING_COUNTRY_NAME,STATE_NAME,CITYVILLAGETOWN_NAME,UUID_,ID_,...,STATUSCOMMENT,LEGACY_BATCH_ID,LEGACY_REPORT_ID,MIGRATED_FLAG,LEGACY_RE_ID,JOB_ID,RCD_CREATE_DATE,RCD_UPDATE_DATE,LEGACY_TRANSACTIONID,NAME
0,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584046336997390,THAILAND,JAMMU AND KASHMIR,SRINAGAR,08909e32-e104-1eda-8d27-e3ef71db905e,9681909,...,None,None,None,None,None,None,None,None,None,None
1,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-384053339983051,NEPAL,JAMMU AND KASHMIR,SRINAGAR,0bf6f3ae-c2ac-587d-65bf-bcf940df7d3c,9729930,...,None,None,None,None,None,None,None,None,None,None
2,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584053275327142,CANADA,JAMMU AND KASHMIR,JAMMU,1c27ddef-7e78-851e-1221-423063769daa,9728773,...,None,None,None,None,None,None,None,None,None,None
3,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584043186303180,CANADA,JAMMU AND KASHMIR,JAMMU,67152aac-2dd1-159f-3e47-2ef9f8e955b5,9654483,...,None,None,None,None,None,None,None,None,None,None
4,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584057377423509,UNITED STATES,JAMMU AND KASHMIR,SRINAGAR,a78c4656-021d-1eb1-315e-d07fae5e5b1c,9720058,...,None,None,None,None,None,None,None,None,None,None


,FIUREID,NAME,FIU_BATCH_ID,YYYY_MM,TRANSACTIONID,ISSUING_COUNTRY_NAME,STATE_NAME,CITYVILLAGETOWN_NAME,UUID_,ID_,...,STATUSCOMMENT,LEGACY_BATCH_ID,LEGACY_REPORT_ID,MIGRATED_FLAG,LEGACY_RE_ID,JOB_ID,RCD_CREATE_DATE,RCD_UPDATE_DATE,LEGACY_TRANSACTIONID,NAME
1,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584046336997390,THAILAND,JAMMU AND KASHMIR,SRINAGAR,08909e32-e104-1eda-8d27-e3ef71db905e,9681909,...,None,None,None,None,None,None,None,None,None,None
2,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-384053339983051,NEPAL,JAMMU AND KASHMIR,SRINAGAR,0bf6f3ae-c2ac-587d-65bf-bcf940df7d3c,9729930,...,None,None,None,None,None,None,None,None,None,None
3,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584053275327142,CANADA,JAMMU AND KASHMIR,JAMMU,1c27ddef-7e78-851e-1221-423063769daa,9728773,...,None,None,None,None,None,None,None,None,None,None
4,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584043186303180,CANADA,JAMMU AND KASHMIR,JAMMU,67152aac-2dd1-159f-3e47-2ef9f8e955b5,9654483,...,None,None,None,None,None,None,None,None,None,None
5,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584057377423509,UNITED STATES,JAMMU AND KASHMIR,SRINAGAR,a78c4656-021d-1eb1-315e-d07fae5e5b1c,9720058,...,None,None,None,None,None,None,None,None,None,None


### (A) -1 MACRO DATA -----------Details of transaction done in India through foreign issued cards

In [9]:
query_1 = f'''

with cte1 as (SELECT distinct coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId ))  report_count,transactiontype,
round(sum(cast([amountInInr]as float) /10000000),4) Withdrawal_in_India
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  and report_type='BDU'
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) = '{x}'
  and state_ in ('14' , '0' ) 
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])), transactiontype
 ),


 cte2 as (
select fiureid, Re_name,--left(fiu_Batch_id,6) as YY_MM ,
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then Withdrawal_in_India  else 0 end) as 'Total_cash_withdrwal_in_India_ATM_Withdrawl',
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then report_count else 0 end) as 'Total_Count_ATM_ATM_Withdrawl',
 sum(Case when transactiontype = 'Charge' then Withdrawal_in_India else 0 end) as 'Total_Transaction_through_POS_in_India_POS',
 sum(Case when transactiontype = 'Charge' then report_count else 0 end) as 'Total_Count_POS'
 from cte1 
 group by fiureid, Re_name--,left(fiu_Batch_id,6)
 )
 select * from cte2 
 
'''

In [10]:
query_1 = pd.read_sql(query_1,FINnet_2_engine)
print('Query Executed successfully')
query_1 = pd.DataFrame(query_1)
query_1.columns = query_1.columns.str.upper()
display("Shape : ",query_1.shape)
display(query_1.head(5))
query_1.index = query_1.index + 1
query_1.head(5)

Query Executed successfully


'Shape : '

(2, 6)

,FIUREID,RE_NAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS
0,FICSO00010,MASTERCARD ASIA/PACIFIC PTE. LIMITED,0.4061,420,2.645,334
1,FICSO00020,VISA WORLDWIDE PTE. LTD,1.4887,1721,8.108,5219


,FIUREID,RE_NAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS
1,FICSO00010,MASTERCARD ASIA/PACIFIC PTE. LIMITED,0.4061,420,2.645,334
2,FICSO00020,VISA WORLDWIDE PTE. LTD,1.4887,1721,8.108,5219


### (A) -2 MACRO DATA -----------------Identification of Foreign Jurisdiction

In [11]:
query_2 = f'''

 with cte1 as (SELECT distinct ts.issuingcountry as CardIssuerCountry,c.name as CardIssuerCountryName ,coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId ))  report_count,transactiontype,
round(sum(cast([amountInInr]as float) /10000000),4) Withdrawal_in_India
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
  where report_type='BDU'
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) = {x}
  and state_ in ('14' , '0' ) 
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])), transactiontype,ts.issuingcountry,c.name
 ),
 --select * from cte1

 cte2 as (
select upper(CardIssuerCountryName) AS CardIssuerCountryName,--CardIssuerCountry,left(fiu_Batch_id,6) as YY_MM ,
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then Withdrawal_in_India  else 0 end) as 'Total_cash_withdrwal_in_India_ATM_Withdrawl',
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then report_count else 0 end) as 'Total_Count_ATM_ATM_Withdrawl',
 sum(Case when transactiontype = 'Charge' then Withdrawal_in_India else 0 end) as 'Total_Transaction_through_POS_in_India_POS',
 sum(Case when transactiontype = 'Charge' then report_count else 0 end) as 'Total_Count_POS' ,
 sum(Case when transactiontype in ('ATM Cash Withdrawal','Charge') then Withdrawal_in_India else 0 end) as 'Cumulative_amount'
 from cte1 
 group by CardIssuerCountryName--,left(fiu_Batch_id,6),CardIssuerCountry
 )

 select * from cte2 order by Cumulative_amount desc
'''

In [12]:
query_2 = pd.read_sql(query_2,FINnet_2_engine)
print('Query Executed successfully')
query_2 = pd.DataFrame(query_2)
query_2.columns = query_2.columns.str.upper()
display("Shape : ",query_2.shape)
display(query_2.head(5))
query_2.index = query_2.index + 1
query_2.head(5)

Query Executed successfully


'Shape : '

(102, 6)

,CARDISSUERCOUNTRYNAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS,CUMULATIVE_AMOUNT
0,UNITED STATES,0.1857,198,2.9418,1241,3.1275
1,UNITED KINGDOM,0.1353,158,2.3785,506,2.5138
2,BANGLADESH,0.2804,346,0.6070,826,0.8874
3,MALAYSIA,0.1284,151,0.6545,645,0.7829
4,UNITED ARAB EMIRATES,0.0633,65,0.4943,135,0.5576


,CARDISSUERCOUNTRYNAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS,CUMULATIVE_AMOUNT
1,UNITED STATES,0.1857,198,2.9418,1241,3.1275
2,UNITED KINGDOM,0.1353,158,2.3785,506,2.5138
3,BANGLADESH,0.2804,346,0.6070,826,0.8874
4,MALAYSIA,0.1284,151,0.6545,645,0.7829
5,UNITED ARAB EMIRATES,0.0633,65,0.4943,135,0.5576


### (A) -3 MACRO DATA ------------------STATE & TOWN WHERE FOREIGN ISSUED CARD IS USED IN INDIA 

In [13]:
query_3 = f'''

 with cte1 as (SELECT distinct coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId )) report_count,transactiontype,
round(sum(cast([amountInInr]as float) /10000000),4) Withdrawal_in_India , ts.cityVillageTown , ts.state_ , ct.name as City_Town_Village , st.name as State_Name
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_State] st (nolock) on st.id_ = ts.state_
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_City] ct (nolock) on ct.id_  = ts.cityVillageTown
  where
  report_type='BDU'
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) = '{x}'
  and state_ in ('14' , '0' ) 
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])), transactiontype,ts.cityVillageTown , ts.state_ , ct.name, st.name
 ),


 cte2 as (
select upper(City_Town_Village) as City_Town_Village ,upper(State_Name) as State_Name ,
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then Withdrawal_in_India  else 0 end) as 'Total_cash_withdrwal_in_India_ATM_Withdrawl',
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then report_count else 0 end) as 'Total_Count_ATM_ATM_Withdrawl',
 sum(Case when transactiontype = 'Charge' then Withdrawal_in_India else 0 end) as 'Total_Transaction_through_POS_in_India_POS',
 sum(Case when transactiontype = 'Charge' then report_count else 0 end) as 'Total_Count_POS',
 sum(Case when transactiontype in ('ATM Cash Withdrawal','Charge') then Withdrawal_in_India else 0 end) as 'Cumulative_amount'
 from cte1 
 group by City_Town_Village ,State_Name 
 )

 select * from cte2 order by Total_cash_withdrwal_in_India_ATM_Withdrawl desc

'''

In [14]:
query_3 = pd.read_sql(query_3,FINnet_2_engine)
print('Query Executed successfully')
query_3 = pd.DataFrame(query_3)
query_3.columns = query_3.columns.str.upper()
display("Shape : ",query_3.shape)
display(query_3.head(5))
query_3.index = query_3.index + 1
query_3.head(5)

Query Executed successfully


'Shape : '

(20, 7)

,CITY_TOWN_VILLAGE,STATE_NAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS,CUMULATIVE_AMOUNT
0,SRINAGAR,JAMMU AND KASHMIR,0.8542,959,5.2557,2955,6.1099
1,None,JAMMU AND KASHMIR,0.4061,420,2.6450,334,3.0511
2,JAMMU,JAMMU AND KASHMIR,0.3032,383,1.1300,919,1.4332
3,LEH,JAMMU AND KASHMIR,0.0939,109,0.3030,203,0.3969
4,ANANTNAG,JAMMU AND KASHMIR,0.0690,79,0.1132,104,0.1822


,CITY_TOWN_VILLAGE,STATE_NAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS,CUMULATIVE_AMOUNT
1,SRINAGAR,JAMMU AND KASHMIR,0.8542,959,5.2557,2955,6.1099
2,None,JAMMU AND KASHMIR,0.4061,420,2.6450,334,3.0511
3,JAMMU,JAMMU AND KASHMIR,0.3032,383,1.1300,919,1.4332
4,LEH,JAMMU AND KASHMIR,0.0939,109,0.3030,203,0.3969
5,ANANTNAG,JAMMU AND KASHMIR,0.0690,79,0.1132,104,0.1822


### CARD LEVEL DATA

### Card level (B -1 ) ---- ATM CASH WITHDRAWL

In [15]:
query_4 = f'''

with cte1 as (SELECT distinct ts.cardNumber ,ts.issuingcountry as CardIssuerCountry,c.name as CardIssuerCountryName, ts.issuingBank ,coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId )) report_count,transactiontype,
sum(cast([amountInInr]as float)) Withdrawal_in_India
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
  where report_type='BDU'
  and transactionType in ('ATM Cash Withdrawal')
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) = '{x}'
  and state_ in ('14' , '0' )
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])) , transactiontype,ts.issuingcountry,c.name , ts.cardNumber , ts.issuingBank 
 ),

 cte2 as (
select FIUREID,RE_NAME,cardNumber, report_count as Distinct_count ,
 sum(Case when transactiontype = 'ATM Cash Withdrawal' then Withdrawal_in_India  else 0 end) as 'Total_cash_withdrwal_in_India_ATM_Withdrawl',
 upper(CardIssuerCountryName) AS CardIssuerCountryName , issuingBank
 from cte1 
 group by CardIssuerCountryName, issuingBank , FIUREID, RE_NAME, cardNumber , report_count
 )

  --select Top 25 * from cte2 order by Total_cash_withdrwal_in_India_ATM_Withdrawl desc

 select  cardNumber , Distinct_count ,round(Total_cash_withdrwal_in_India_ATM_Withdrawl/10000000,4) as Total_cash_withdrwal_in_India_ATM_Withdrawl ,CardIssuerCountryName , issuingBank , RE_NAME from (
 select Top 25 * from cte2 order by Total_cash_withdrwal_in_India_ATM_Withdrawl desc ) z
 
 '''

In [16]:
query_4 = pd.read_sql(query_4,FINnet_2_engine)
print('Query Executed successfully')
query_4 = pd.DataFrame(query_4)
query_4.columns = query_4.columns.str.upper()
display("Shape : ",query_4.shape)
display(query_4.head(5))
query_4.index = query_4.index + 1
query_4.head(5)

Query Executed successfully


'Shape : '

(25, 6)

,CARDNUMBER,DISTINCT_COUNT,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,CARDISSUERCOUNTRYNAME,ISSUINGBANK,RE_NAME
0,535550XXXXXX1883,58,0.0580,LITHUANIA,None,MASTERCARD ASIA/PACIFIC PTE. LIMITED
1,519892XXXXXX0347,36,0.0364,SOUTH KOREA,None,MASTERCARD ASIA/PACIFIC PTE. LIMITED
2,416202XXXXXX4461,33,0.0316,THAILAND,KASIKORNBANK PUBLIC CO LTD,VISA WORLDWIDE PTE. LTD
3,422174XXXXXX0351,30,0.0301,GERMANY,DEUTSCHE KREDITBANK AG,VISA WORLDWIDE PTE. LTD
4,511879XXXXXX7003,21,0.0210,UNITED ARAB EMIRATES,None,MASTERCARD ASIA/PACIFIC PTE. LIMITED


,CARDNUMBER,DISTINCT_COUNT,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,CARDISSUERCOUNTRYNAME,ISSUINGBANK,RE_NAME
1,535550XXXXXX1883,58,0.0580,LITHUANIA,None,MASTERCARD ASIA/PACIFIC PTE. LIMITED
2,519892XXXXXX0347,36,0.0364,SOUTH KOREA,None,MASTERCARD ASIA/PACIFIC PTE. LIMITED
3,416202XXXXXX4461,33,0.0316,THAILAND,KASIKORNBANK PUBLIC CO LTD,VISA WORLDWIDE PTE. LTD
4,422174XXXXXX0351,30,0.0301,GERMANY,DEUTSCHE KREDITBANK AG,VISA WORLDWIDE PTE. LTD
5,511879XXXXXX7003,21,0.0210,UNITED ARAB EMIRATES,None,MASTERCARD ASIA/PACIFIC PTE. LIMITED


### Card level (B -1 ) -------- CHARGE

In [17]:
query_5 = f'''
 with cte1 as (SELECT distinct ts.cardNumber ,ts.issuingcountry as CardIssuerCountry,c.name as CardIssuerCountryName, ts.issuingBank ,coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId )) report_count,transactiontype,
sum(cast([amountInInr]as float)) Withdrawal_in_India
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
  where report_type='BDU'
  and transactionType in ('Charge')
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) = '{x}'
  and state_ in ('14' , '0' )
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])) , transactiontype,ts.issuingcountry,c.name , ts.cardNumber , ts.issuingBank 
 ),
-- select * from cte1


 cte2 as (
select FIUREID,RE_NAME,cardNumber, report_count as Distinct_Count,
 sum(Case when transactiontype = 'Charge' then Withdrawal_in_India else 0 end) as 'Total_Transaction_through_POS_in_India_POS'
 ,upper(CardIssuerCountryName) AS CardIssuerCountryName , issuingBank
 from cte1 
 group by CardIssuerCountryName, issuingBank , FIUREID, RE_NAME, cardNumber , report_count
 )

  --select Top 25 * from cte2 order by Cumulative_Amount desc

 select  cardNumber, Distinct_Count ,round(Total_Transaction_through_POS_in_India_POS/10000000,4) as Total_Transaction_through_POS_in_India_POS , CardIssuerCountryName , issuingBank , RE_NAME from (
 select Top 25 * from cte2 order by Total_Transaction_through_POS_in_India_POS desc ) z
'''

In [18]:
query_5 = pd.read_sql(query_5,FINnet_2_engine)
print('Query Executed successfully')
query_5 = pd.DataFrame(query_5)
query_5.columns = query_5.columns.str.upper()
display("Shape : ",query_5.shape)
display(query_5.head(5))
query_5.index = query_5.index + 1
query_5.head(5)

Query Executed successfully


'Shape : '

(25, 6)

,CARDNUMBER,DISTINCT_COUNT,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,CARDISSUERCOUNTRYNAME,ISSUINGBANK,RE_NAME
0,459632XXXXXX9019,5,0.3490,UNITED KINGDOM,BARCLAYS BANK PLC,VISA WORLDWIDE PTE. LTD
1,414720XXXXXX6460,3,0.1517,UNITED STATES,JPMORGAN CHASE BANK NA,VISA WORLDWIDE PTE. LTD
2,438857XXXXXX6829,3,0.1317,UNITED STATES,JPMORGAN CHASE BANK NA,VISA WORLDWIDE PTE. LTD
3,465911XXXXXX6019,11,0.1179,UNITED KINGDOM,BARCLAYS BANK UK PLC,VISA WORLDWIDE PTE. LTD
4,417749XXXXXX8154,3,0.1048,KYRGYZSTAN,OJSC COMMERCIAL BANK KYRGYZSTA,VISA WORLDWIDE PTE. LTD


,CARDNUMBER,DISTINCT_COUNT,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,CARDISSUERCOUNTRYNAME,ISSUINGBANK,RE_NAME
1,459632XXXXXX9019,5,0.3490,UNITED KINGDOM,BARCLAYS BANK PLC,VISA WORLDWIDE PTE. LTD
2,414720XXXXXX6460,3,0.1517,UNITED STATES,JPMORGAN CHASE BANK NA,VISA WORLDWIDE PTE. LTD
3,438857XXXXXX6829,3,0.1317,UNITED STATES,JPMORGAN CHASE BANK NA,VISA WORLDWIDE PTE. LTD
4,465911XXXXXX6019,11,0.1179,UNITED KINGDOM,BARCLAYS BANK UK PLC,VISA WORLDWIDE PTE. LTD
5,417749XXXXXX8154,3,0.1048,KYRGYZSTAN,OJSC COMMERCIAL BANK KYRGYZSTA,VISA WORLDWIDE PTE. LTD


### Card level (B -3 ) -------- ATM CASH WITHDRAWL - Trend past 6 month

In [19]:
def adjustmonth_(month):
    if month%100 == 0:
        first_two_month = month//100
        month = (first_two_month-1)*100+12
    return month

In [20]:
curr_m = x
curr_m_1 = x-1 
curr_m_1 = adjustmonth_(curr_m_1)
curr_m_2 = curr_m_1 - 1
curr_m_2 = adjustmonth_(curr_m_2)
curr_m_3 = curr_m_2 - 1
curr_m_3 = adjustmonth_(curr_m_3)
curr_m_4 = curr_m_3 - 1
curr_m_4 = adjustmonth_(curr_m_4)
curr_m_5 = curr_m_4 - 1
curr_m_5 = adjustmonth_(curr_m_5)
curr_m_6 = curr_m_5 - 1
curr_m_6 = adjustmonth_(curr_m_6)


In [21]:
print('Curr_m  ',curr_m)
print('curr_m_1',curr_m_1)
print('curr_m_2',curr_m_2)
print('curr_m_3',curr_m_3)
print('curr_m_4',curr_m_4)
print('curr_m_5',curr_m_5)
print('curr_m_6',curr_m_6)

Curr_m   202403
curr_m_1 202402
curr_m_2 202401
curr_m_3 202312
curr_m_4 202311
curr_m_5 202310
curr_m_6 202309


In [22]:
query_6 = f'''
  
 with cte1 as (SELECT  ts.cardNumber ,ts.issuingcountry as CardIssuerCountry,c.name as CardIssuerCountryName, ts.issuingBank ,coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId )) report_count,left(fiu_Batch_id,6) AS fiu_Batch_id ,transactiontype,
 sum(cast([amountInInr]as float)) Withdrawal_in_India , ts.state_ as statecode
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
  where report_type='BDU'
  and transactionType in ('ATM Cash Withdrawal')
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) between '{curr_m_6 }' and '{curr_m}'
  and state_ in ('14' , '0' )
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])), fiu_Batch_id, transactiontype,ts.issuingcountry,c.name , ts.cardNumber , ts.issuingBank , ts.state_

 
 ),


cte2 as (

 select Top 25 cardNumber,Amount_Mar, Amount_Feb, Amount_Jan , Amount_Dec , Amount_Nov , Amount_Oct , Amount_Sept from 
 (select distinct cte1.cardNumber,
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m}' then Withdrawal_in_India  end) as 'Amount_Mar',
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m_1}' then Withdrawal_in_India  end) as 'Amount_Feb',
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m_2}' then Withdrawal_in_India  end) as 'Amount_Jan',
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m_3}' then Withdrawal_in_India  end) as 'Amount_Dec',
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m_4}' then Withdrawal_in_India  end) as 'Amount_Nov',
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m_5}' then Withdrawal_in_India  end) as 'Amount_Oct',
 sum(Case when transactiontype in ('ATM Cash Withdrawal' ) and fiu_Batch_id = '{curr_m_6}' then Withdrawal_in_India  end) as 'Amount_Sept'
 from cte1
 group by cardNumber)z
 where Amount_Mar is not null
 order by Amount_Mar desc


 )

 
select distinct cardNumber,round(Amount_Mar/10000000,4) curr_m   ,round(Amount_Feb/10000000,4) curr_m_1 , round(Amount_Jan/10000000,4) curr_m_2 , round(Amount_Dec/10000000,4) curr_m_3 , round(Amount_Nov/10000000,4) curr_m_4 , round(Amount_Oct/10000000,4) curr_m_5 , round(Amount_Sept/10000000,4)  curr_m_6 
from cte2 order by curr_m desc


'''

In [23]:
query_6 = pd.read_sql(query_6,FINnet_2_engine)
print('Query Executed successfully')
query_6 = pd.DataFrame(query_6)
query_6.columns = query_6.columns.str.upper()
display("Shape : ",query_6.shape)
display(query_6.head(5))
query_6.index = query_6.index + 1
query_6.head(5)

Query Executed successfully


'Shape : '

(25, 8)

,CARDNUMBER,CURR_M,CURR_M_1,CURR_M_2,CURR_M_3,CURR_M_4,CURR_M_5,CURR_M_6
0,535550XXXXXX1883,0.0580,0.2450,None,0.1166,0.0180,NaN,None
1,519892XXXXXX0347,0.0364,0.0556,None,0.0572,0.0240,NaN,None
2,416202XXXXXX4461,0.0316,0.0093,None,NaN,0.0063,0.0053,None
3,422174XXXXXX0351,0.0301,0.0140,None,0.0150,0.0120,0.0140,None
4,511879XXXXXX7003,0.0210,0.0505,None,0.0601,0.0930,NaN,None


,CARDNUMBER,CURR_M,CURR_M_1,CURR_M_2,CURR_M_3,CURR_M_4,CURR_M_5,CURR_M_6
1,535550XXXXXX1883,0.0580,0.2450,None,0.1166,0.0180,NaN,None
2,519892XXXXXX0347,0.0364,0.0556,None,0.0572,0.0240,NaN,None
3,416202XXXXXX4461,0.0316,0.0093,None,NaN,0.0063,0.0053,None
4,422174XXXXXX0351,0.0301,0.0140,None,0.0150,0.0120,0.0140,None
5,511879XXXXXX7003,0.0210,0.0505,None,0.0601,0.0930,NaN,None


### Card level (B -3 ) -------- CHARGE - Trend past 6 month

In [24]:
query_7 = f'''

with cte1 as (SELECT  ts.cardNumber ,ts.issuingcountry as CardIssuerCountry,c.name as CardIssuerCountryName, ts.issuingBank ,coalesce([oldREId],[newREId]) as FIUREID,trim(upper(re.[name])) as RE_NAME, COUNT(distinct concat(Transactionid,'|',reportId )) report_count,left(fiu_Batch_id,6) AS fiu_Batch_id ,transactiontype,
 sum(cast([amountInInr]as float)) Withdrawal_in_India , ts.state_ as statecode
  FROM [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
  join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_
  join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts (nolock) on cp.Report_Id=ts.reportId
  left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
  where report_type='BDU'
  and transactionType in ('Charge')
  and cp.Report_Status = 'Pass'
  and left(cp.Fiu_Batch_Id,6) between '{curr_m_6 }' and '{curr_m}'
  and state_ in ('14' , '0' )
 group by coalesce([oldREId],[newREId]),trim(upper(re.[name])), fiu_Batch_id, transactiontype,ts.issuingcountry,c.name , ts.cardNumber , ts.issuingBank , ts.state_

 
 ),


cte2 as (

 select Top 25 cardNumber,Amount_Mar, Amount_Feb, Amount_Jan , Amount_Dec , Amount_Nov , Amount_Oct , Amount_Sept from 
 (select distinct cte1.cardNumber,
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m}' then Withdrawal_in_India  end) as 'Amount_Mar',
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m_1}' then Withdrawal_in_India  end) as 'Amount_Feb',
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m_2}' then Withdrawal_in_India  end) as 'Amount_Jan',
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m_3}' then Withdrawal_in_India  end) as 'Amount_Dec',
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m_4}' then Withdrawal_in_India  end) as 'Amount_Nov',
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m_5}' then Withdrawal_in_India  end) as 'Amount_Oct',
 sum(Case when transactiontype in ('Charge' ) and fiu_Batch_id = '{curr_m_6}' then Withdrawal_in_India  end) as 'Amount_Sept'
 from cte1
 group by cardNumber)z
 where Amount_Mar is not null
 order by Amount_Mar desc


 )

 
select distinct cardNumber,round(Amount_Mar/10000000,4) curr_m   ,round(Amount_Feb/10000000,4) curr_m_1 , round(Amount_Jan/10000000,4) curr_m_2 , round(Amount_Dec/10000000,4) curr_m_3 , round(Amount_Nov/10000000,4) curr_m_4 , round(Amount_Oct/10000000,4) curr_m_5 , round(Amount_Sept/10000000,4)  curr_m_6 
from cte2 order by curr_m desc



'''

In [25]:
query_7 = pd.read_sql(query_7,FINnet_2_engine)
print('Query Executed successfully')
query_7 = pd.DataFrame(query_7)
query_7.columns = query_7.columns.str.upper()
display("Shape : ",query_7.shape)
display(query_7.head(5))
query_7.index = query_7.index + 1
query_7.head(5)

Query Executed successfully


'Shape : '

(25, 8)

,CARDNUMBER,CURR_M,CURR_M_1,CURR_M_2,CURR_M_3,CURR_M_4,CURR_M_5,CURR_M_6
0,459632XXXXXX9019,0.3490,NaN,None,NaN,NaN,NaN,None
1,414720XXXXXX6460,0.1517,NaN,None,NaN,NaN,NaN,None
2,438857XXXXXX6829,0.1317,NaN,None,NaN,NaN,NaN,None
3,465911XXXXXX6019,0.1179,0.6665,None,0.0691,NaN,NaN,None
4,417749XXXXXX8154,0.1048,NaN,None,NaN,NaN,NaN,None


,CARDNUMBER,CURR_M,CURR_M_1,CURR_M_2,CURR_M_3,CURR_M_4,CURR_M_5,CURR_M_6
1,459632XXXXXX9019,0.3490,NaN,None,NaN,NaN,NaN,None
2,414720XXXXXX6460,0.1517,NaN,None,NaN,NaN,NaN,None
3,438857XXXXXX6829,0.1317,NaN,None,NaN,NaN,NaN,None
4,465911XXXXXX6019,0.1179,0.6665,None,0.0691,NaN,NaN,None
5,417749XXXXXX8154,0.1048,NaN,None,NaN,NaN,NaN,None


### NEW POINT ------------ Merchant Name

### (i) -----ATM CASH WITHDRAWL

In [26]:

query_8 = f'''

with cte1 as (

select upper(trim(merchantName)) as merchantName ,round(sum(cast([amountInInr]as float))/10000000,4) as Total_Amount
 from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
 inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts on ts.reportId = cp.Report_Id
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_ 
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
 where transactionType = 'ATM Cash Withdrawal' and Report_Type = 'BDU' and state_ in ('14' , '0')
 and LEFT(Fiu_Batch_Id,6) = '{x}'
 group by upper(trim(merchantName))

),


cte11 as (select merchantName , STRING_AGG ( merchantId, '--' ) as Merchant_ID from(
select distinct merchantName , concat('M-',merchantId)merchantId  from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on cp.Report_Id = ts.reportId
where left(Fiu_Batch_Id,6) = '{x}' and state_ in ('14','0') and  ts.reportType = 'BDU' and merchantId is not null) z
group by merchantName
),


cte2 as (
select coalesce([oldREId],[newREId]) as FIUREID, re.name as RE_Name  , upper(trim(merchantName)) as merchantName , c.name as CardIssuerCountryName , issuingBank ,cardNumber , round(sum(cast([amountInInr]as float))/10000000,4) as Card_Level_Amount 
 from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
 inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts on ts.reportId = cp.Report_Id
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_ 
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
 where transactionType = 'ATM Cash Withdrawal' and Report_Type = 'BDU' and state_ in ('14' , '0')
 and LEFT(Fiu_Batch_Id,6) = '{x}'
 group by coalesce([oldREId],[newREId]), re.name ,upper(trim(merchantName)) , c.name ,  issuingBank , ts.cardNumber
)




select upper(trim(c2.RE_Name))RE_Name , upper(trim(c2.merchantName)) as merchantName , c1.Total_Amount , upper(trim(c2.CardIssuerCountryName))CardIssuerCountryName , c2.issuingBank ,  c2.cardNumber , Merchant_ID ,c2.Card_Level_Amount from cte2 c2
inner join cte1 c1 on  upper(trim(c2.merchantName)) = upper(trim(c1.merchantName))
left join cte11 on upper(trim(cte11.merchantName)) = upper(trim(c2.merchantName)) 
order by Total_Amount desc
 
 
'''

In [27]:
query_8 = pd.read_sql(query_8,FINnet_2_engine)
print('Query Executed successfully')
query_8 = pd.DataFrame(query_8)
query_8.columns = query_8.columns.str.upper()
display("Shape : ",query_8.shape)
display(query_8.head(5))
query_8.index = query_8.index + 1
query_8.head(5)

Query Executed successfully


'Shape : '

(1007, 8)

,RE_NAME,MERCHANTNAME,TOTAL_AMOUNT,CARDISSUERCOUNTRYNAME,ISSUINGBANK,CARDNUMBER,MERCHANT_ID,CARD_LEVEL_AMOUNT
0,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,BRAC BANK LIMITED,477792XXXXXX1076,M-50431583--M-79394577--M-12371805,0.001
1,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,IFIC BANK PLC,458790XXXXXX2816,M-50431583--M-79394577--M-12371805,0.001
2,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,UNITED COMMERCIAL BANK,462870XXXXXX8597,M-50431583--M-79394577--M-12371805,0.001
3,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,EASTERN BANK PLC,423800XXXXXX4440,M-50431583--M-79394577--M-12371805,0.001
4,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,CITY BANK PLC,410520XXXXXX2163,M-50431583--M-79394577--M-12371805,0.001


,RE_NAME,MERCHANTNAME,TOTAL_AMOUNT,CARDISSUERCOUNTRYNAME,ISSUINGBANK,CARDNUMBER,MERCHANT_ID,CARD_LEVEL_AMOUNT
1,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,BRAC BANK LIMITED,477792XXXXXX1076,M-50431583--M-79394577--M-12371805,0.001
2,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,IFIC BANK PLC,458790XXXXXX2816,M-50431583--M-79394577--M-12371805,0.001
3,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,UNITED COMMERCIAL BANK,462870XXXXXX8597,M-50431583--M-79394577--M-12371805,0.001
4,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,EASTERN BANK PLC,423800XXXXXX4440,M-50431583--M-79394577--M-12371805,0.001
5,VISA WORLDWIDE PTE. LTD,DAL LAKE SRINAGAR OATM,0.0971,BANGLADESH,CITY BANK PLC,410520XXXXXX2163,M-50431583--M-79394577--M-12371805,0.001


### NEW POINT ------------ Merchant Name

### (ii) -----Charge

In [28]:

query_9 = f'''

with cte1 as (

select  upper(trim(merchantName)) as merchantName ,round(sum(cast([amountInInr]as float))/10000000,4) as Total_Amount
 from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
 inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts on ts.reportId = cp.Report_Id
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_ 
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
 where transactionType = 'Charge' and Report_Type = 'BDU' and state_ in ('14' , '0')
 and LEFT(Fiu_Batch_Id,6) = '{x}'
 group by upper(trim(merchantName))

),


cte11 as (select merchantName , STRING_AGG ( merchantId, '--' ) as Merchant_ID from(
select distinct merchantName , concat('M-',merchantId)merchantId   from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp on cp.Report_Id = ts.reportId
where left(Fiu_Batch_Id,6) = '{x}' and state_ in ('14','0') and  ts.reportType = 'BDU' and merchantId is not null) z
group by merchantName
),


cte2 as (
select coalesce([oldREId],[newREId]) as FIUREID, re.name as RE_Name  , upper(trim(merchantName)) as merchantName , c.name as CardIssuerCountryName , issuingBank ,cardNumber , round(sum(cast([amountInInr]as float))/10000000,4) as Card_Level_Amount 
 from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp  (nolock)
 inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts on ts.reportId = cp.Report_Id
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on cp.Re_Id = re.id_ 
 left join [DATAHUB_DB].[FIUMetaHub].[finnet_Country] c (nolock) on c.id_ = ts.issuingcountry
 where transactionType = 'Charge' and Report_Type = 'BDU' and state_ in ('14' , '0')
 and LEFT(Fiu_Batch_Id,6) = '{x}'
 group by coalesce([oldREId],[newREId]), re.name ,upper(trim(merchantName)) , c.name ,  issuingBank , ts.cardNumber
)




select upper(trim(c2.RE_Name))RE_Name , upper(trim(c2.merchantName)) as merchantName , c1.Total_Amount , upper(trim(c2.CardIssuerCountryName))CardIssuerCountryName , c2.issuingBank ,  c2.cardNumber , Merchant_ID ,c2.Card_Level_Amount from cte2 c2
inner join cte1 c1 on  upper(trim(c2.merchantName)) = upper(trim(c1.merchantName))
left join cte11 on upper(trim(cte11.merchantName)) = upper(trim(c2.merchantName)) 
order by Total_Amount desc
 

'''

In [29]:
query_9 = pd.read_sql(query_9,FINnet_2_engine)
print('Query Executed successfully')
query_9 = pd.DataFrame(query_9)
query_9.columns = query_9.columns.str.upper()
display("Shape : ",query_9.shape)
display(query_9.head(5))
query_9.index = query_9.index + 1
query_9.head(5)

Query Executed successfully


'Shape : '

(4619, 8)

,RE_NAME,MERCHANTNAME,TOTAL_AMOUNT,CARDISSUERCOUNTRYNAME,ISSUINGBANK,CARDNUMBER,MERCHANT_ID,CARD_LEVEL_AMOUNT
0,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,AUSTRALIA,ANZ BANKING GROUP LTD,464579XXXXXX1095,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0055
1,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,MALAYSIA,CIMB BANK BERHAD,462888XXXXXX1397,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0023
2,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,UNITED ARAB EMIRATES,EMIRATES NBD BANK PJSC,467745XXXXXX1334,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0165
3,MASTERCARD ASIA/PACIFIC PTE. LIMITED,HIMALAYA SPORTS,0.2994,PHILIPPINES,None,518869XXXXXX1986,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0216
4,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,AUSTRALIA,ANZ BANKING GROUP LTD,462239XXXXXX1055,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0015


,RE_NAME,MERCHANTNAME,TOTAL_AMOUNT,CARDISSUERCOUNTRYNAME,ISSUINGBANK,CARDNUMBER,MERCHANT_ID,CARD_LEVEL_AMOUNT
1,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,AUSTRALIA,ANZ BANKING GROUP LTD,464579XXXXXX1095,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0055
2,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,MALAYSIA,CIMB BANK BERHAD,462888XXXXXX1397,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0023
3,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,UNITED ARAB EMIRATES,EMIRATES NBD BANK PJSC,467745XXXXXX1334,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0165
4,MASTERCARD ASIA/PACIFIC PTE. LIMITED,HIMALAYA SPORTS,0.2994,PHILIPPINES,None,518869XXXXXX1986,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0216
5,VISA WORLDWIDE PTE. LTD,HIMALAYA SPORTS,0.2994,AUSTRALIA,ANZ BANKING GROUP LTD,462239XXXXXX1055,M-0341257A0006918--M-19872248--M-0341257A0014173,0.0015


In [30]:
df = [query_anomalie,raw_data , query_1 , query_2 , query_3 , query_4 , query_5 , query_6 , query_7 , query_8 , query_9 ]

In [31]:
df[4]

,CITY_TOWN_VILLAGE,STATE_NAME,TOTAL_CASH_WITHDRWAL_IN_INDIA_ATM_WITHDRAWL,TOTAL_COUNT_ATM_ATM_WITHDRAWL,TOTAL_TRANSACTION_THROUGH_POS_IN_INDIA_POS,TOTAL_COUNT_POS,CUMULATIVE_AMOUNT
1,SRINAGAR,JAMMU AND KASHMIR,0.8542,959,5.2557,2955,6.1099
2,None,JAMMU AND KASHMIR,0.4061,420,2.6450,334,3.0511
3,JAMMU,JAMMU AND KASHMIR,0.3032,383,1.1300,919,1.4332
4,LEH,JAMMU AND KASHMIR,0.0939,109,0.3030,203,0.3969
5,ANANTNAG,JAMMU AND KASHMIR,0.0690,79,0.1132,104,0.1822
6,BARAMULA,JAMMU AND KASHMIR,0.0388,41,0.7582,360,0.7970
7,GANDERBAL,JAMMU AND KASHMIR,0.0382,42,0.1506,66,0.1888
8,UDHAMPUR,JAMMU AND KASHMIR,0.0173,21,0.0151,18,0.0324
9,REASI,JAMMU AND KASHMIR,0.0156,19,0.0050,17,0.0206
10,BADGAM,JAMMU AND KASHMIR,0.0149,16,0.1966,158,0.2115


In [32]:
query_anomalie

,PARAMETER,RE_NAME,MANDATORY,TOTAL_COUNT,MISSING_RECORDS,ANOMALIES_PERCENTAGE
1,ADDRESS LINE 1,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
2,ADDRESS LINE 1,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
3,CITY/VILLAGE/TOWN,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
4,DISTRICT,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
5,DISTRICT,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
6,ISSUING BANK,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
7,LOCALITY,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
8,LOCALITY,VISA WORLDWIDE PTE. LTD,N,6940,6940,100%
9,MERCHANT CATEGORY CODE,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%
10,MERCHANT COUNTRY,MASTERCARD ASIA/PACIFIC PTE. LIMITED,N,754,754,100%


In [115]:
format1_df = query_anomalie
format2_df = pd.pivot_table(format1_df,
                            index=['PARAMETER', 'MANDATORY'],
                            columns='RE_NAME',
                            values=['TOTAL_COUNT', 'MISSING_RECORDS'],
                            aggfunc={'TOTAL_COUNT': 'sum',
                                     'MISSING_RECORDS': 'sum'},
                            fill_value=max(TOTAL_COUNT),
#                             margins=True,
#                             margins_name='Grand Total'
                           )

format2_df = format2_df.sort_index(axis=1, level='RE_NAME')
format2_df=format2_df.reset_index()
format2_df
# Flatten multi-index columns
format2_df.columns = [' '.join(col).strip() for col in format2_df.columns.values]
display(format2_df)

NameError: name 'TOTAL_COUNT' is not defined

In [54]:
format2_df = format2_df.reset_index().rename(columns={
    'Total_count M': 'Sum of Total_count (M)',
    'Total_count V': 'Sum of Total_count (V)',
    'Missing_records M': 'Sum of Missing_records (M)',
    'Missing_records V': 'Sum of Missing_records (V)',
    'anomalies_percentage M': 'Sum of anomalies_percentage (M)',
    'anomalies_percentage V': 'Sum of anomalies_percentage (V)',
    'Total_count': 'Sum of Total_count',
    'Missing_records': 'Sum of Missing_records',
    'anomalies_percentage': 'Sum of anomalies_percentage'
})

# Reorder columns to match Format2
format2_df = format2_df[['Parameter', 'Mandatory',
                         'Sum of Total_count (M)', 'Sum of Missing_records (M)', 'Sum of anomalies_percentage (M)',
                         'Sum of Total_count (V)', 'Sum of Missing_records (V)', 'Sum of anomalies_percentage (V)']]

# Print or return the transformed DataFrame
# print(format2_df)
format2_df

KeyError: "None of [Index(['Parameter', 'Mandatory', 'Sum of Total_count (M)',\n       'Sum of Missing_records (M)', 'Sum of anomalies_percentage (M)',\n       'Sum of Total_count (V)', 'Sum of Missing_records (V)',\n       'Sum of anomalies_percentage (V)'],\n      dtype='object')] are in the [columns]"

In [ ]:
import pandas as pd
format1_df=pd.read_csv(r'\\172.20.1.68\shared\dipto_data\try1.csv')
format1_df
# Pivot the DataFrame to get Format2
format2_df = pd.pivot_table(format1_df,
                            index=['Parameter', 'Mandatory'],
                            columns='name',
                            values=['Total_count', 'Missing_records', 'anomalies_percentage'],
                            aggfunc={'Total_count': 'sum',
                                     'Missing_records': 'sum',
                                     'anomalies_percentage': 'sum'},
                            fill_value=0,
                            margins=True,
                            margins_name='Grand Total')

# Flatten multi-index columns
format2_df.columns = [' '.join(col).strip() for col in format2_df.columns.values]

# Rename columns to match Format2
format2_df = format2_df.reset_index().rename(columns={
    'Total_count M': 'Sum of Total_count (M)',
    'Total_count V': 'Sum of Total_count (V)',
    'Missing_records M': 'Sum of Missing_records (M)',
    'Missing_records V': 'Sum of Missing_records (V)',
    'anomalies_percentage M': 'Sum of anomalies_percentage (M)',
    'anomalies_percentage V': 'Sum of anomalies_percentage (V)',
    'Total_count': 'Sum of Total_count',
    'Missing_records': 'Sum of Missing_records',
    'anomalies_percentage': 'Sum of anomalies_percentage'
})

# Reorder columns to match Format2
format2_df = format2_df[['Parameter', 'Mandatory',
                         'Sum of Total_count (M)', 'Sum of Missing_records (M)', 'Sum of anomalies_percentage (M)',
                         'Sum of Total_count (V)', 'Sum of Missing_records (V)', 'Sum of anomalies_percentage (V)']]

# Print or return the transformed DataFrame
# print(format2_df)
format2_df

In [83]:
raw_data.head(5)

,FIUREID,NAME,FIU_BATCH_ID,YYYY_MM,TRANSACTIONID,ISSUING_COUNTRY_NAME,STATE_NAME,CITYVILLAGETOWN_NAME,UUID_,ID_,...,STATUSCOMMENT,LEGACY_BATCH_ID,LEGACY_REPORT_ID,MIGRATED_FLAG,LEGACY_RE_ID,JOB_ID,RCD_CREATE_DATE,RCD_UPDATE_DATE,LEGACY_TRANSACTIONID,NAME
1,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584046336997390,THAILAND,JAMMU AND KASHMIR,SRINAGAR,08909e32-e104-1eda-8d27-e3ef71db905e,9681909,...,None,None,None,None,None,None,None,None,None,None
2,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-384053339983051,NEPAL,JAMMU AND KASHMIR,SRINAGAR,0bf6f3ae-c2ac-587d-65bf-bcf940df7d3c,9729930,...,None,None,None,None,None,None,None,None,None,None
3,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584053275327142,CANADA,JAMMU AND KASHMIR,JAMMU,1c27ddef-7e78-851e-1221-423063769daa,9728773,...,None,None,None,None,None,None,None,None,None,None
4,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584043186303180,CANADA,JAMMU AND KASHMIR,JAMMU,67152aac-2dd1-159f-3e47-2ef9f8e955b5,9654483,...,None,None,None,None,None,None,None,None,None,None
5,FICSO00020,Visa Worldwide Pte. Ltd,ID-20240312140335696,202403,T-584057377423509,UNITED STATES,JAMMU AND KASHMIR,SRINAGAR,a78c4656-021d-1eb1-315e-d07fae5e5b1c,9720058,...,None,None,None,None,None,None,None,None,None,None


In [84]:
query_trn_type = f'''select coalesce(oldREId,newreid) as FIUREID , re.name as RE_NAME , ts.transactionType , round(sum(cast([amountInInr]as float) /10000000),4) Withdrawal_in_India , count(distinct(concat(ts.reportId,'|',ts.transactionId))) as Unique_count from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] cp 
inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_TsEleven] ts on ts.reportId = cp.Report_Id
inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re on re.id_ = cp.Re_Id
where cp.Report_Type = 'BDU' and cp.Report_Status = 'PASS' and LEFT(Fiu_Batch_Id,6) = '{x}' and state_ in ('14','0')
group by coalesce(oldREId,newreid) , re.name , ts.transactionType'''

In [85]:
query_trn_type = pd.read_sql(query_trn_type,FINnet_2_engine)
print('Query Executed successfully')
query_trn_type = pd.DataFrame(query_trn_type)
query_trn_type.columns = query_trn_type.columns.str.upper()
display("Shape : ",query_trn_type.shape)
display(query_trn_type.head(5))
query_trn_type.index = query_trn_type.index + 1
query_trn_type.head(5)

Query Executed successfully


'Shape : '

(4, 5)

,FIUREID,RE_NAME,TRANSACTIONTYPE,WITHDRAWAL_IN_INDIA,UNIQUE_COUNT
0,FICSO00020,Visa Worldwide Pte. Ltd,Charge,8.1080,5219
1,FICSO00020,Visa Worldwide Pte. Ltd,ATM Cash Withdrawal,1.4887,1721
2,FICSO00010,MASTERCARD ASIA/PACIFIC PTE. LIMITED,Charge,2.6450,334
3,FICSO00010,MASTERCARD ASIA/PACIFIC PTE. LIMITED,ATM Cash Withdrawal,0.4061,420


,FIUREID,RE_NAME,TRANSACTIONTYPE,WITHDRAWAL_IN_INDIA,UNIQUE_COUNT
1,FICSO00020,Visa Worldwide Pte. Ltd,Charge,8.1080,5219
2,FICSO00020,Visa Worldwide Pte. Ltd,ATM Cash Withdrawal,1.4887,1721
3,FICSO00010,MASTERCARD ASIA/PACIFIC PTE. LIMITED,Charge,2.6450,334
4,FICSO00010,MASTERCARD ASIA/PACIFIC PTE. LIMITED,ATM Cash Withdrawal,0.4061,420


In [99]:
format1_trn = query_trn_type
# print(format1_df.head(5))
format1_trn = pd.pivot_table(format1_trn,
                            index=['TRANSACTIONTYPE'],
                            columns='RE_NAME',
                            values=['UNIQUE_COUNT', 'WITHDRAWAL_IN_INDIA'],
                            aggfunc={'UNIQUE_COUNT': 'sum',
                                     'WITHDRAWAL_IN_INDIA': 'sum'},
#                             fill_value=0,
#                             margins=True,
#                             margins_name='Grand Total'
                           )

format1_trn = format1_trn.sort_index(axis=1, level='RE_NAME')
format1_trn.head(5)

# Flatten multi-index columns
format1_trn.columns = [' '.join(col).strip() for col in format1_trn.columns.values]
display(format1_trn)



,UNIQUE_COUNT MASTERCARD ASIA/PACIFIC PTE. LIMITED,WITHDRAWAL_IN_INDIA MASTERCARD ASIA/PACIFIC PTE. LIMITED,UNIQUE_COUNT Visa Worldwide Pte. Ltd,WITHDRAWAL_IN_INDIA Visa Worldwide Pte. Ltd
TRANSACTIONTYPE,,,,
ATM Cash Withdrawal,420,0.4061,1721,1.4887
Charge,334,2.6450,5219,8.1080


In [101]:
format1_trn.reset_index()

,TRANSACTIONTYPE,UNIQUE_COUNT MASTERCARD ASIA/PACIFIC PTE. LIMITED,WITHDRAWAL_IN_INDIA MASTERCARD ASIA/PACIFIC PTE. LIMITED,UNIQUE_COUNT Visa Worldwide Pte. Ltd,WITHDRAWAL_IN_INDIA Visa Worldwide Pte. Ltd
0,ATM Cash Withdrawal,420,0.4061,1721,1.4887
1,Charge,334,2.6450,5219,8.1080
